In [1]:
import os
from warnings import filterwarnings
from dotenv import load_dotenv, find_dotenv
filterwarnings(action="ignore")

from operator import itemgetter
from collections import defaultdict

from langchain_groq import ChatGroq
from langchain_core.messages import trim_messages, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory, RunnablePassthrough

load_dotenv(find_dotenv())

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

#### Initialize model

In [3]:
groq_model = ChatGroq(model="llama-3.3-70b-versatile")
groq_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000018002A158B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018002A16390>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

#### Adding Prompt to model

In [5]:
model_prompt = ChatPromptTemplate(messages=[
    ("system", "You're a helpful Q/A conversational bot. Answer each questions to the best of your capability."),
    MessagesPlaceholder(variable_name="user_input"),
    # ("user", "{user_input}")
])

model_prompt

ChatPromptTemplate(input_variables=['user_input'], input_types={'user_input': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core

#### Adding Conversation history

In [6]:
class BotMemory:
    def __init__(self):
        self.history = defaultdict(ChatMessageHistory)

    def GetMessageHistory(self, chat_id:str)->ChatMessageHistory:
        """
        This helps to retrieve or initialize a session's chat message history.
        """
        return self.history[chat_id]

In [7]:
bot_memory = BotMemory()

#### Managing Chat Database

In [8]:
message_trimmer = trim_messages(max_tokens=128000,
                                strategy="last",
                                token_counter=groq_model,
                                start_on="human",
                                include_system=True,
                                allow_partial=False,)

message_trimmer

RunnableLambda(...)

#### Creating a Langchain Expression Language i.e chain for the prompt and the model

In [9]:
model_chain = (RunnablePassthrough(messages=itemgetter("messages")|message_trimmer) 
               | model_prompt 
               | groq_model)

model_chain

RunnablePassthrough()
| ChatPromptTemplate(input_variables=['user_input'], input_types={'user_input': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.

#### Putting it all together

In [10]:
bot_with_memory = RunnableWithMessageHistory(model_chain, bot_memory.GetMessageHistory)

bot_with_memory

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<bound method BotMemory.GetMessageHistory of <__main__.BotMemory object at 0x000001807FEABB90>>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

#### Run Chatbot

In [12]:
# while True:
#     user_input = input("User:")
#     if user_input != "end":                
#         bot_response = bot_with_memory.invoke(
#                                               {"user_input": [HumanMessage(content=user_input)]}, 
#                                               config={"configurable": {"session_id": "Eddy"}}
#                                               ).content
#         print(f"Bot: {bot_response}", end="\n\n")
#     else:
#         break